In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm  
from dataloader import *
from model import * 
from nltk.translate.bleu_score import sentence_bleu
import nltk 
import torch.nn as nn
import torch.nn.functional as F


# Hyperparameters
num_epochs = 10
learning_rate = 0.001
target_confidence = 0.8 

In [31]:
def collate_fn(batch):
    inputs, labels = zip(*batch)
    max_length = max(len(seq) for seq in inputs)
    
    # Convert each sequence to a list, pad with 0, and convert to tensor
    padded_inputs = [torch.cat([seq, torch.zeros(max_length - len(seq), dtype=torch.long)]) for seq in inputs]
    lengths = [len(seq) for seq in inputs]
    
    return torch.stack(padded_inputs), torch.tensor(labels, dtype=torch.float), lengths

def tokens_to_words(token_ids, vocab):
    inv_vocab = {v: k for k, v in vocab.items()}
    return [inv_vocab.get(token_id, '<UNK>') for token_id in token_ids if token_id != 0]  # Exclude padding


class TextDatasetTest(Dataset):
    def __init__(self, data_dir, vocab, files):
        super(TextDatasetTest, self).__init__()
        self.data = []
        self.vocab = vocab 

        for filename in files:
            file_path = os.path.join(data_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                for line in lines:
                    tokens = line.strip().split()
                    label = 1 if filename.endswith('.1') else 0  # Binary label
                    self.data.append((tokens, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens, label = self.data[idx]
        token_ids = [self.vocab.get(token, self.vocab['<UNK>']) for token in tokens]
        return torch.tensor(token_ids, dtype=torch.long), torch.tensor(label, dtype=torch.long)
     
    
files = ["sentiment.train.0", "sentiment.train.1"]
files_test = ["sentiment.test.0", "sentiment.test.1"]

data_dir = "./data/sentiment_style_transfer/yelp"
vocab = build_vocab(data_dir)
dataset = TextDatasetTest(data_dir, vocab, files)
dataset_test = TextDatasetTest(data_dir, vocab, files_test)
data_loader = DataLoader(dataset, batch_size=64, collate_fn=collate_fn, shuffle=True)
data_loader_test = DataLoader(dataset_test, batch_size=64, collate_fn=collate_fn, shuffle=False)

In [23]:
len(dataset)

443259

In [24]:
class DisentangledVAE(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, style_dim, content_dim):
        super(DisentangledVAE, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_dim = hidden_dim
        
        # Encoder
        self.encoder_rnn = nn.GRU(embedding_dim, hidden_dim, 
                                 batch_first=True, 
                                 bidirectional=True,
                                 num_layers=2,
                                 dropout=0.2)
        
        # Latent spaces
        self.style_encoder = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.content_encoder = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Mean and logvar projections
        self.style_mu = nn.Linear(hidden_dim, style_dim)
        self.style_logvar = nn.Linear(hidden_dim, style_dim)
        self.content_mu = nn.Linear(hidden_dim, content_dim)
        self.content_logvar = nn.Linear(hidden_dim, content_dim)
        
        # Decoder
        self.latent_to_hidden = nn.Sequential(
            nn.Linear(style_dim + content_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        self.decoder_rnn = nn.GRU(embedding_dim + hidden_dim, hidden_dim,
                                 batch_first=True,
                                 num_layers=2,
                                 dropout=0.2)
        
        self.output_fc = nn.Linear(hidden_dim, vocab_size)
        
        # Style classifier for adversarial training
        self.style_classifier = nn.Sequential(
            nn.Linear(style_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
    def encode(self, x, lengths=None):
        batch_size = x.size(0)
        
        # Embed input
        embedded = self.embedding(x)
        
        # Pack for variable length sequences
        if lengths is not None:
            embedded = nn.utils.rnn.pack_padded_sequence(
                embedded, lengths, batch_first=True, enforce_sorted=False
            )
        
        # Encode
        _, hidden = self.encoder_rnn(embedded)
        # Combine bidirectional states
        hidden = torch.cat([hidden[-2], hidden[-1]], dim=1)
        
        # Encode style and content
        style_hidden = self.style_encoder(hidden)
        content_hidden = self.content_encoder(hidden)
        
        # Get latent parameters
        style_mu = self.style_mu(style_hidden)
        style_logvar = self.style_logvar(style_hidden)
        content_mu = self.content_mu(content_hidden)
        content_logvar = self.content_logvar(content_hidden)
        
        return style_mu, style_logvar, content_mu, content_logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
        
    def decode(self, style, content, x):
        batch_size = x.size(0)
        max_len = x.size(1)
        
        # Combine latent vectors
        latent = torch.cat([style, content], dim=1)
        hidden = self.latent_to_hidden(latent)
        
        # Initialize decoder hidden state
        hidden = hidden.unsqueeze(0).repeat(2, 1, 1)  # num_layers * batch * hidden
        
        # Teacher forcing with concatenated latent
        embedded = self.embedding(x)
        hidden_expanded = hidden[-1].unsqueeze(1).repeat(1, max_len, 1)
        decoder_input = torch.cat([embedded, hidden_expanded], dim=2)
        
        # Decode
        outputs, _ = self.decoder_rnn(decoder_input, hidden)
        outputs = self.output_fc(outputs)
        
        return outputs

    def forward(self, x, lengths=None):
        # Encode
        style_mu, style_logvar, content_mu, content_logvar = self.encode(x, lengths)
        
        # Sample latent vectors
        style = self.reparameterize(style_mu, style_logvar)
        content = self.reparameterize(content_mu, content_logvar)
        
        # Decode
        recon_x = self.decode(style, content, x)
        
        return recon_x, style_mu, style_logvar, content_mu, content_logvar, style, content

    def classify_style(self, style):
        return self.style_classifier(style)

def vae_loss(recon_x, x, style_mu, style_logvar, content_mu, content_logvar):
    recon_loss = F.cross_entropy(recon_x.view(-1, recon_x.size(-1)), x.view(-1), ignore_index=0)  # Reconstruction loss
    kl_style = -0.5 * torch.sum(1 + style_logvar - style_mu.pow(2) - style_logvar.exp())  # KL divergence for style
    kl_content = -0.5 * torch.sum(1 + content_logvar - content_mu.pow(2) - content_logvar.exp())  # KL divergence for content
    return recon_loss + kl_style + kl_content

def multi_task_loss(style_preds, style_labels, content_preds, content_labels):
    style_loss = F.cross_entropy(style_preds, style_labels)  # Style classification loss
    content_loss = F.cross_entropy(content_preds, content_labels)  # Content classification loss
    return style_loss + content_loss

def adversarial_loss(style_preds, content_preds):
    adversarial_style_loss = -F.cross_entropy(style_preds, torch.zeros_like(style_preds))  # Fool style classifier
    adversarial_content_loss = -F.cross_entropy(content_preds, torch.zeros_like(content_preds))  # Fool content classifier
    return adversarial_style_loss + adversarial_content_loss


In [25]:
len(data_loader)

6926

In [26]:
def train_vae_with_dataset(vae, optimizer, data_loader, device):
    """
    Train the VAE with a custom dataset.
    """
    vae.train()
    total_loss = 0
    num_batches = len(data_loader)
    
    print(f"\nTraining on {len(data_loader.dataset)} examples with {num_batches} batches")
    
    # Progress bar
    progress = tqdm(data_loader, total=num_batches, desc="Training")
    
    batch_count = 0
    for input_tokens, style_labels, lengths in progress:
        batch_count += 1
        batch_size = input_tokens.size(0)
        
        # Log batch information
        if batch_count == 1:
            print(f"\nFirst batch shapes:")
            print(f"Input tokens: {input_tokens.shape}")
            print(f"Style labels: {style_labels.shape}")
            print(f"Sequence lengths: {lengths[:5]} (showing first 5)")
        
        # Move data to device
        input_tokens = input_tokens.to(device)
        style_labels = style_labels.to(device)
        
        # Forward pass
        recon_x, style_mu, style_logvar, content_mu, content_logvar, style, content = vae(input_tokens, lengths)
        
        # Calculate losses
        loss_vae = F.cross_entropy(recon_x.view(-1, recon_x.size(-1)), input_tokens.view(-1), ignore_index=0)
        kl_style = -0.5 * torch.sum(1 + style_logvar - style_mu.pow(2) - style_logvar.exp())
        kl_content = -0.5 * torch.sum(1 + content_logvar - content_mu.pow(2) - content_logvar.exp())
        
        style_preds = vae.classify_style(style).squeeze()
        style_labels = style_labels.float()
        loss_multi_task = F.binary_cross_entropy(style_preds, style_labels)
        
        # Total loss with coefficient scaling
        loss = loss_vae + 0.1 * (kl_style + kl_content) + loss_multi_task
        
        # Optimization
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(vae.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Update progress bar with current loss
        progress.set_postfix({'batch_loss': f'{loss.item():.4f}'})
        total_loss += loss.item()
        
        # Detailed loss logging every 100 batches
        if batch_count % 100 == 0:
            print(f"\nBatch {batch_count}/{num_batches}")
            print(f"Reconstruction Loss: {loss_vae.item():.4f}")
            print(f"KL Loss: {(kl_style + kl_content).item():.4f}")
            print(f"Style Loss: {loss_multi_task.item():.4f}")
    
    avg_loss = total_loss / num_batches
    print(f"\nCompleted epoch. Average loss: {avg_loss:.4f}")
    return avg_loss

# Training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(vocab)
embedding_dim = 256
hidden_dim = 512
style_dim = 32
content_dim = 256
learning_rate = 5e-4
epochs = 10

# Initialize model and optimizer
vae = DisentangledVAE(vocab_size, embedding_dim, hidden_dim, style_dim, content_dim).to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)

print(f"Starting training for {epochs} epochs...")
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    print("-" * 50)
    loss = train_vae_with_dataset(vae, optimizer, data_loader, device)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")


Starting training for 10 epochs...

Epoch 1/10
--------------------------------------------------

Training on 443259 examples with 6926 batches


Training:   0%|          | 9/6926 [00:00<01:17, 89.28it/s, batch_loss=7.0265]


First batch shapes:
Input tokens: torch.Size([64, 15])
Style labels: torch.Size([64])
Sequence lengths: [6, 6, 11, 2, 7] (showing first 5)


Training:   2%|▏         | 116/6926 [00:01<01:08, 99.26it/s, batch_loss=2.5323]


Batch 100/6926
Reconstruction Loss: 2.4905
KL Loss: 0.1018
Style Loss: 0.6910


...


Batch 1700/6926
Reconstruction Loss: 0.0000
KL Loss: 0.0001
Style Loss: 0.6915


Training:  26%|██▌       | 1812/6926 [00:18<00:51, 99.81it/s, batch_loss=0.6641]


Batch 1800/6926
Reconstruction Loss: 0.0000
KL Loss: 0.0001
Style Loss: 0.6789


Training:  27%|██▋       | 1878/6926 [00:18<00:49, 100.98it/s, batch_loss=0.6355]


KeyboardInterrupt: 

In [27]:
class StyleClassifier(nn.Module):
    def _init_(self, vocab_size, embedding_dim, hidden_dim):
        super(StyleClassifier, self)._init_()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.GRU(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.embedding(x)
        _, hidden = self.encoder(x)
        # Combine bidirectional states
        hidden = torch.cat([hidden[-2], hidden[-1]], dim=1)
        return self.fc(hidden)

#### UPDATED PART ABOVE, NOT UPDATED PART BELOW

In [28]:
torch.save(vae, 'model_complete.pth')

In [29]:
model_1 = torch.load('model_complete.pth') 

/tmp/ipykernel_146017/3363134432.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_1 = torch.load('model_complete.pth')


In [34]:


# Inspect some sentences from the data loader 
model_1.eval()
with torch.no_grad():
    for input_tokens, _, lengths in data_loader_test:
        input_tokens = input_tokens.to(device)
        x_reconstructed, _, _, _, _, _, _  = model_1(input_tokens)
        x_reconstructed = x_reconstructed.argmax(dim=-1)  # Get the predicted token IDs

        # Print a few input and output sentences
        for i in range(5):  # Print 5 examples
            original_sentence = tokens_to_words(input_tokens[i].tolist(), vocab)
            reconstructed_sentence = tokens_to_words(x_reconstructed[i].tolist(), vocab)

            print("Original Sentence: \t\t\t", " ".join(original_sentence))
            print("Reconstructed Sentence: \t\t", " ".join(reconstructed_sentence))
            print()

        break  # Only inspect the first batch

Original Sentence: 			 ever since joes has changed hands it 's just gotten worse and worse .
Reconstructed Sentence: 		 ever since joes has changed hands it 's just gotten worse and worse . tasting

Original Sentence: 			 there is definitely not enough room in that part of the venue .
Reconstructed Sentence: 		 there is definitely not enough room in that part of the venue . tasting tasting

Original Sentence: 			 so basically tasted watered down .
Reconstructed Sentence: 		 so basically tasted watered down . tasting tasting tasting tasting tasting tasting tasting tasting tasting

Original Sentence: 			 she said she 'd be back and disappeared for a few minutes .
Reconstructed Sentence: 		 she said she 'd be back and disappeared for a few minutes . tasting joining

Original Sentence: 			 i ca n't believe how inconsiderate this pharmacy is .
Reconstructed Sentence: 		 i ca n't believe how inconsiderate this pharmacy is . tasting joining joining joining joining



In [33]:
def train_style_classifier(data_loader, vocab_size, device):
    classifier = StyleClassifier(vocab_size, 300, 128).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

    classifier.train()
    for epoch in range(20):  # Train for a few epochs
        total_loss = 0
        for input_tokens, labels, _ in data_loader:  # Adjusted to unpack three values
            input_tokens = input_tokens.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            predictions = classifier(input_tokens)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/5, Loss: {total_loss / len(data_loader)}")
    
    return classifier

def evaluate_style_transfer(data_loader, model, classifier, device):
    model.eval()
    classifier.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for input_tokens, labels, _ in data_loader:
            input_tokens = input_tokens.to(device)
            labels = labels.to(device)

            # Get the reconstructed sentences
            x_reconstructed, _, _, _, _, _, _ = model(input_tokens)
            x_reconstructed = x_reconstructed.argmax(dim=-1)

            # Predict the style of the reconstructed sentences
            style_predictions = classifier(x_reconstructed)
            style_labels = (style_predictions > 0.5).float()
            
            correct_predictions += (style_labels == labels).sum().item()
            total_predictions += labels.size(0)
    
    accuracy = correct_predictions / total_predictions
    print(f"Style Transfer Accuracy: {accuracy:.4f}")

In [17]:
classifier = train_style_classifier(data_loader, len(vocab), device)
evaluate_style_transfer(data_loader_test, model_1, classifier, device)

KeyboardInterrupt: 

In [18]:
#### updated code 
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')

def tokens_to_words(token_ids, vocab):
    inv_vocab = {v: k for k, v in vocab.items()}
    return [inv_vocab.get(token_id, '<UNK>') for token_id in token_ids if token_id != 0]

def calculate_bleu_score(data_loader, model, vocab, device):
    model.eval()
    total_bleu_score = 0
    num_sentences = 0
    smoothing_fn = SmoothingFunction().method1

    print("\nBLEU-S: Evaluating content preservation...\n")
    with torch.no_grad():
        for input_tokens, _, lengths in data_loader:
            input_tokens = input_tokens.to(device)
            x_reconstructed, _, _, _ = model(input_tokens)
            x_reconstructed = x_reconstructed.argmax(dim=-1)

            for i in range(min(5, len(input_tokens))):  
                original_sentence = tokens_to_words(input_tokens[i].tolist(), vocab)
                reconstructed_sentence = tokens_to_words(x_reconstructed[i].tolist(), vocab)
                print(f"Original: {' '.join(original_sentence)}")
                print(f"Reconstructed: {' '.join(reconstructed_sentence)}\n")

                bleu_score = sentence_bleu([original_sentence], reconstructed_sentence, smoothing_function=smoothing_fn)
                total_bleu_score += bleu_score
                num_sentences += 1

            break  # Evaluate only on the first batch for now

    avg_bleu_score = total_bleu_score / num_sentences if num_sentences > 0 else 0
    print(f"Average BLEU-S Score: {avg_bleu_score:.4f}")
    return avg_bleu_score

def train_style_classifier(data_loader, vocab_size, device):
    classifier = StyleClassifier(vocab_size, 300, 128).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

    classifier.train()
    print("\nTraining Style Classifier...\n")
    for epoch in range(20):
        total_loss = 0
        for input_tokens, labels, _ in data_loader:
            input_tokens = input_tokens.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            predictions = classifier(input_tokens)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/5, Loss: {total_loss / len(data_loader):.4f}")
    
    return classifier

def evaluate_style_transfer(data_loader, model, classifier, vocab, device):
    model.eval()
    classifier.eval()
    correct_predictions = 0
    total_predictions = 0

    print("\nEvaluating Style Transfer Accuracy...\n")
    with torch.no_grad():
        for input_tokens, labels, _ in data_loader:
            input_tokens = input_tokens.to(device)
            labels = labels.to(device)

            x_reconstructed, _, _, _ = model(input_tokens)
            x_reconstructed = x_reconstructed.argmax(dim=-1)

            style_predictions = classifier(x_reconstructed)
            style_labels = (style_predictions > 0.5).float()
            correct_predictions += (style_labels == labels).sum().item()
            total_predictions += labels.size(0)

            for i in range(min(5, len(input_tokens))):
                original_sentence = tokens_to_words(input_tokens[i].tolist(), vocab)
                reconstructed_sentence = tokens_to_words(x_reconstructed[i].tolist(), vocab)
                print(f"Original: {' '.join(original_sentence)}")
                print(f"Reconstructed: {' '.join(reconstructed_sentence)}")
                print(f"Style Prediction: {style_labels[i].item()}, True Style: {labels[i].item()}\n")

            break  # Evaluate only on the first batch for now

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    print(f"Style Transfer Accuracy: {accuracy:.4f}")
    return accuracy

def run_evaluation(data_loader_train, data_loader_test, model, vocab, vocab_size, device):
    # Train Style Classifier
    classifier = train_style_classifier(data_loader_train, vocab_size, device)

    print("\n--- BLEU-S Score (Content Preservation) ---")
    bleu_score = calculate_bleu_score(data_loader_test, model, vocab, device)

    print("\n--- Style Transfer Accuracy ---")
    style_transfer_accuracy = evaluate_style_transfer(data_loader_test, model, classifier, vocab, device)

    print("\n--- Final Results ---")
    print(f"BLEU-S Score: {bleu_score:.4f}")
    print(f"Style Transfer Accuracy: {style_transfer_accuracy:.4f}")

    return bleu_score, style_transfer_accuracy

run_evaluation(data_loader, data_loader_test, model, vocab, len(vocab), device)

[nltk_data] Downloading package punkt to /home/qik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Training Style Classifier...

Epoch 1/5, Loss: 0.0954
Epoch 2/5, Loss: 0.0522
Epoch 3/5, Loss: 0.0387
Epoch 4/5, Loss: 0.0300
Epoch 5/5, Loss: 0.0241
Epoch 6/5, Loss: 0.0205
Epoch 7/5, Loss: 0.0178
Epoch 8/5, Loss: 0.0158
Epoch 9/5, Loss: 0.0145
Epoch 10/5, Loss: 0.0139
Epoch 11/5, Loss: 0.0132
Epoch 12/5, Loss: 0.0124
Epoch 13/5, Loss: 0.0116
Epoch 14/5, Loss: 0.0117
Epoch 15/5, Loss: 0.0116
Epoch 16/5, Loss: 0.0113
Epoch 17/5, Loss: 0.0113
Epoch 18/5, Loss: 0.0108
Epoch 19/5, Loss: 0.0105
Epoch 20/5, Loss: 0.0104

--- BLEU-S Score (Content Preservation) ---

BLEU-S: Evaluating content preservation...

Original: at this location the service was terrible .
Reconstructed: so brand scale crumbs crappy steady shogun shogun adds adds adds adds adds adds adds

Original: i ordered garlic bread and fettuccine alfredo pasta with vegetables .
Reconstructed: anyways a+ cancer crusted cancer middle towing crusted middle watered belgian camarones inspector inspector inspector

Original: i did n't

(0.0, 0.546875)

In [20]:
for a, b, c in data_loader:
    for i in a:
        print(i.shape)

torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15])
torch.Size([15